In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm, metrics
from tqdm import tqdm_notebook as tqdm
import random
from sklearn.dummy import DummyClassifier
from scipy import stats
import math
from skimage import morphology, img_as_bool

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path_to_data = "gdrive/My Drive/data/train_images.npy"
path_to_labels = "gdrive/My Drive/data/train_labels.csv"
path_to_test = "gdrive/My Drive/data/test_images.npy"

In [0]:
def plot_images(list_of_images, max_col = 4):
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0], cmap="gray"); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(20, max_col * r))
        for i, (img,name) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title(str(name))
            ax.axis('off')
            ax.imshow(img, cmap="gray")

In [0]:
def get_data(path_x, path_y):
    data = np.load(path_x, encoding = 'bytes')
    labels_df = pd.read_csv(path_y)
    labels_df.Category = pd.Categorical(labels_df.Category)
    y = labels_df.Category.cat.codes.values
    X = np.array(data[:, 1])
    for c, i in enumerate(X):
        ret,thresh_img = cv2.threshold(i,127,255,cv2.THRESH_BINARY)
        X[c] = thresh_img
    return X, y

In [6]:
X_noisy, y = get_data(path_to_data, path_to_labels)
print("Shape of training dataset:", len(X_noisy))
X_test = list(np.load(path_to_test, encoding = 'bytes')[:, 1])
print("Shape of testing dataset:", len(X_test))
unique_classes = set(y)
print("Number of classes:", len(unique_classes))

Shape of training dataset: 10000
Shape of testing dataset: 10000
Number of classes: 31


In [0]:
from scipy import ndimage

def remove_noise(filledImg, min_size = 500):
    blobs, min_val = ndimage.label(filledImg)
    
    for i in range(10,60):
        clean_img = morphology.remove_small_objects(img_as_bool(filledImg), i)
        blobs, number_of_blobs = ndimage.label(clean_img)
        if number_of_blobs < min_val:
            min_val = number_of_blobs
            if min_val == 1:
                return clean_img
    return clean_img

In [8]:
X = []
for c, img in enumerate(X_noisy):
    img = img.reshape(100, 100)
    X.append(remove_noise(img, 100).flatten())
  

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type float64 to positive image of type bool.
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to bool
  .format(dtypeobj_in, dtypeobj_out))


In [0]:
for c, img in enumerate(X):
    X[c] = np.array(img)
X = np.array(X)

In [50]:
def crop_image(array_X, size_of_image = 100, size_of_crop = 32, show_image = False):
  cropped = []
  cropBox = None
  for i in range(len(array_X)):
    test = array_X[i]
    test.resize(size_of_image,size_of_image)
    image_data = test
    if cropBox == None:
      prev_cropBox = None
    else:
      prev_cropBox = cropBox[:]

    try:  
      non_empty_columns = np.where(image_data.max(axis=0)>0)[0]
      non_empty_rows = np.where(image_data.max(axis=1)>0)[0]
      cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))
    except:
      cropBox = prev_cropBox
    shiftX = size_of_crop
    shiftY = size_of_crop
    cropBox = list(cropBox)
    if(cropBox[0] >= 5):
      cropBox[0] -= 5

    if(cropBox[0]+shiftX >= size_of_image):
      cropBox[0] = size_of_image - shiftX - 1

    if(cropBox[2] >= 5):
      cropBox[2] -= 5

    if(cropBox[2]+shiftY >= size_of_image):
      cropBox[2] = size_of_image - shiftY - 1

    image_data_new = image_data[cropBox[0]:cropBox[0]+shiftX, cropBox[2]:cropBox[2]+shiftY]
    
    if show_image:
      plot_images(np.array([image_data_new]))

    cropped.append(image_data_new.flatten())
  
  for c, img in enumerate(cropped):
    cropped[c] = np.array(img)
  cropped = np.array(cropped)
  return cropped

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [0]:
X_cropped = crop_image(X, size_of_crop=32)

In [76]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
num_classes = 31
epochs = 50

# input image dimensions
img_rows, img_cols = 32, 32
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = (X_cropped[:7000],y[:7000]), (X_cropped[7000:],y[7000:]) 
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[3])
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(x_train)
val_datagen = ImageDataGenerator()
val_datagen.fit(x_test)
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(x_test, y_test, batch_size=batch_size)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

optimizer = keras.optimizers.Nadam(lr=0.002,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)

# Set our optimizer and loss function (similar settings to our CAE approach)
model.compile(loss = keras.losses.categorical_crossentropy,
            optimizer = optimizer,
            metrics = ['categorical_accuracy'])

model.fit_generator(train_generator,
          steps_per_epoch= 14000 // batch_size,
          epochs=epochs,
          verbose=1)

# model.fit(x_train, y_train,
#           batch_size= batch_size,
#           epochs=epochs,
#           verbose=1,
#          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (7000, 32, 32, 1)
7000 train samples
3000 test samples
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
Epoch 1/50
437/437 [==============================] - 13s 29ms/step - loss: 2.7785 - categorical_accuracy: 0.1992
Epoch 2/50
437/437 [==============================] - 10s 24ms/step - loss: 2.2449 - categorical_accuracy: 0.3501
Epoch 3/50
437/437 [==============================] - 10s 24ms/step - loss: 1.9841 - categorical_accuracy: 0.4236
Epoch 4/50
437/437 [==============================] - 10s 24ms/step - loss: 1.8478 - categorical_accuracy: 0.4629
Epoch 5/50
437/437 [==============================] - 10s 24ms/step - loss: 1.7455 - categorical_accuracy: 0.4935
Epoch 6/50
437/437 [==============================] - 10s 24ms/step - loss: 1.6687 - categorical_accuracy: 0.5101
Epoch 7/50
437/437 [==============================] - 10s 24ms/step - loss: 1.6019 - categorical_accuracy: 0.5345
Epoch 8/50
437/437 [================

In [79]:
test = []
for c, img in enumerate(X_test):
    img = img.reshape(100, 100)
    test.append(remove_noise(img, 100).flatten())
    
for c, img in enumerate(test):
    test[c] = np.array(img)
test = np.array(test)


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:118: UserWarning: Possible sign loss when converting negative image of type float64 to positive image of type bool.
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to bool
  .format(dtypeobj_in, dtypeobj_out))


In [0]:
test_cropped = crop_image(test)

In [81]:
test_x = test_cropped
if K.image_data_format() == 'channels_first':
    test_x = test_x.reshape(test_x.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    test_x = test_x.reshape(test_x.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
test_x = test_x.astype('float32')

prediction = model.predict_classes(test_x, batch_size=1, verbose=1)

10000/10000 [==============================] - 25s 2ms/step


In [0]:
import pandas as pd
result = []
labels_df = pd.read_csv(path_to_labels)
labels_df.Category = pd.Categorical(labels_df.Category)
cat_map = pd.Categorical(labels_df.Category)
cat_map = cat_map.categories
for index, image in enumerate(prediction):
  result.append(cat_map[image])
  
result = pd.DataFrame(result)
result = result.reset_index(drop=False)
result.columns=['Id', 'Category']
result.to_csv('result.csv', index=False)